<a href="https://colab.research.google.com/github/jyryu3161/lec_bioai/blob/main/featurization_protein.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
file_uploaded = files.upload()

# 입력 파일 설정

In [4]:
input_file = 'protein_seq_input.csv' # 입력 파일명만 입력

## 환경 구성

In [5]:
!pip install torch transformers sentencepiece h5py

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 프로그램 실행

In [28]:
import pandas as pd
import numpy as np
import os

def one_hot_encoding(sequence):
    vocab = ['A', 'C', 'D', 'E', 
         'F', 'G', 'H', 'I', 
         'K', 'L', 'M', 'N', 
         'P', 'Q', 'R', 'S',
         'T', 'V', 'W', 'Y']

    feature_vectors = []
    for i in range(len(sequence)):
        zero_vector = np.zeros(len(vocab))
        aa = sequence[i]
        for j in range(len(vocab)):
            if aa == vocab[j]:
                zero_vector[j] = 1

        feature_vectors+=list(zero_vector)
    return feature_vectors


############### one-hot 계산 ###############

X_list = []
y_list = []
protein_seq_list = []

with open(input_file, 'r') as fp:
    fp.readline()
    for line in fp:
        sptlist = line.strip().split(',')
        try:
            seq = sptlist[0].strip()
            label = float(sptlist[1].strip())
            feature = one_hot_encoding(seq)
            
            X_list.append(feature)
            y_list.append(label)
            protein_seq_list.append(seq)
        except:
            pass
X_list = np.asarray(X_list)
y_list = np.asarray(y_list)
protein_seq_list = np.asarray(protein_seq_list)

tmp_cols = []
for i in range(X_list.shape[1]):
    tmp_cols.append('X_%s'%(i+1))

df_tmp = pd.DataFrame(X_list, columns=tmp_cols)
df_tmp['label'] = y_list
df_tmp['protein_seq'] = protein_seq_list

output_file_onehot = 'output_onehot_%s'%(os.path.basename(input_file))
df_tmp.to_csv(output_file_onehot, index=False)

########################################################################

import torch
from transformers import AutoTokenizer, AutoModel, pipeline
import re
import numpy as np
import os
import requests
from tqdm.auto import tqdm

tokenizer = AutoTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = AutoModel.from_pretrained("Rostlab/prot_bert")
fe = pipeline('feature-extraction', model=model, tokenizer=tokenizer,device=0)


X_list = []
y_list = []
protein_seq_list = []

with open(input_file, 'r') as fp:
    fp.readline()
    for line in fp:
        sptlist = line.strip().split(',')
        seq = sptlist[0].strip()
        label = float(sptlist[1].strip())

        tmp_list = []
        for c in seq:
            tmp_list.append(c)
        new_seq = ' '.join(tmp_list)
        label = float(sptlist[1].strip())
        protein_seq_list.append(new_seq)
        y_list.append(label)

protein_seq_list = protein_seq_list
embedding = fe(protein_seq_list)
embedding = np.array(embedding)

X_list = []
for seq_num in range(len(protein_seq_list)):
    feature = embedding[seq_num][0].mean(0)
    X_list.append(feature)

X_list = np.asarray(X_list)
y_list = np.asarray(y_list)
protein_seq_list = np.asarray(protein_seq_list)

tmp_cols = []
for i in range(X_list.shape[1]):
    tmp_cols.append('X_%s'%(i+1))

df_tmp = pd.DataFrame(X_list, columns=tmp_cols)
df_tmp['label'] = y_list
df_tmp['protein_seq'] = protein_seq_list

output_file_onehot = 'output_bert_%s'%(os.path.basename(input_file))
df_tmp.to_csv(output_file_onehot, index=False)

Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


(36391, 1, 12, 1024)
36391


AttributeError: ignored